# DO NOT RUN

이 노트북은 참고용 코드 스니펫만 포함하고 있습니다. 워크샵 중에 이 노트북을 실행하지 마세요.

## 제어권 반환을 통한 대화형 에이전트 흐름: 실행 시간 동안 추가 입력을 위한 사용자 상호작용

때로는 도구를 실행하거나 상위 수준의 작업을 해결하기 위해 추가 입력이 필요할 수 있습니다. 이 경우, 사용자 피드백을 수집하기 위해 제어권을 사용자에게 반환해야 합니다. LangGraph에서는 이를 브레이크포인트와 같은 개념으로 구현할 수 있습니다: 특정 단계에서 그래프 실행을 중지합니다. 이 브레이크포인트에서 사용자 입력을 기다릴 수 있습니다. 사용자로부터 입력을 받으면 그래프 상태에 추가하고 진행할 수 있습니다. 다음에서는 제어권 반환을 통한 사용자 상호작용을 지원하도록 에이전트 어시스턴트를 확장할 것입니다.

### 추가 도구: AskHuman

사용자를 흐름에 참여시키기 위해서는, 별도의 도구를 만들어야 합니다. 이를 `AskHuman`이라고 부릅니다.

In [ ]:
from pydantic import BaseModel

class AskHuman(BaseModel):
    """Ask missing information from the user"""

    question: str

## 에이전트

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig

primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant capable of providing travel recommendations."
            " Use the provided tools to look for personalized travel recommendations and information about specific destinations."
            " If you dont have enough information then use AskHuman tool to get required information. "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            " If a search comes up empty, expand your search before giving up."
            " If you dont have enough information then use AskHuman tool to get required information. ",
        ),
        ("placeholder", "{messages}"),
    ]
)

llm = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0,
    max_tokens=None,
    client=bedrock_client,
    # other params...
)

runnable_with_tools = primary_assistant_prompt | llm.bind_tools(tools + [AskHuman])
def call_model(state: State, config: RunnableConfig):
    response = runnable_with_tools.invoke(state)
    return {"messages": [response]}

사용자에게 질문하기 위한 fake 노드도 정의해야 합니다

In [ ]:
# We define a fake node to ask the human
def ask_human(state):
    pass

엣지에 대한 조건부 라우팅을 처리할 수 있는 함수도 정의해보겠습니다.

In [ ]:
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    elif last_message.tool_calls[0]["name"] == "AskHuman":
        return "ask_human"
    # Otherwise if there is, we continue
    else:
        return "continue"

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder = StateGraph(State)

# Define nodes: these do the work
graph_builder.add_edge(START, "assistant")
graph_builder.add_node("assistant", Assistant(runnable_with_tools))
graph_builder.add_node("tools", ToolNode(tools=tools))
graph_builder.add_node("ask_human", ask_human)

# Define edges: these determine how the control flow moves
graph_builder.add_conditional_edges(
    "assistant",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # We may ask the human
        "ask_human": "ask_human",
        # Otherwise we finish.
        "end": END,
    },
)

graph_builder.add_edge("tools", "assistant")
# After we get back the human response, we go back to the agent
graph_builder.add_edge("ask_human", "assistant")

# The checkpointer lets the graph persist its state
# this is a complete memory for the entire graph.
memory = MemorySaver()
agent_with_hil = graph_builder.compile(
    checkpointer=memory, interrupt_before=["ask_human"]
)

In [ ]:
from IPython.display import Image, display

display(Image(agent_with_hil.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}

input_message = HumanMessage(content="I want to book a travel destination")
for event in agent_with_hil.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    event["messages"][-1].pretty_print()

이제 사용자 입력을 흐름에 전달하려고 합니다. 따라서 이 스레드의 상태를 사용자의 응답으로 업데이트해야 합니다. AskHuman을 도구 호출로 취급하고 있으므로, 해당 도구 호출의 ID를 포함하여 도구 호출 응답 스타일로 상태를 업데이트해야 합니다.

In [ ]:
tool_call_id = (
    agent_with_hil.get_state(config).values["messages"][-1].tool_calls[0]["id"]
)

# We now create the tool call with the id and the response we want
tool_message = [
    {"tool_call_id": tool_call_id, "type": "tool", "content": "I love beaches!"}
]

agent_with_hil.update_state(config, {"messages": tool_message}, as_node="ask_human")

agent_with_hil.get_state(config).next

이제 사용자 입력을 상태에 주입했습니다. `.next` 함수는 상태 그래프에서 정의한 대로 워크플로우 `실행`의 다음 단계가 어시스턴트가 될 것임을 명확하게 보여줍니다. 이제 그래프 `실행`을 계속할 수 있습니다.

In [ ]:
for event in agent_with_hil.stream(None, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

## 중단 후 상태 업데이트가 있는 호텔 에이전트

다음 코드 스니펫에서는 민감한 도구에 대해 실행을 중단하고 실행을 계속하기 위한 사용자 승인 후에 상태를 업데이트하는 방법을 보여줄 것입니다.

우리의 커스텀 호텔 에이전트를 만들기 위해 그래프에 모든 노드를 추가하고 컴파일해보겠습니다.

이 그래프는 다음을 포함하는 호텔 예약 시스템의 흐름을 정의할 것입니다:

1. 요청을 처리하기 위한 메인 호텔 에이전트 노드
2. 호텔 예약 검색 및 조회를 실행하기 위한 도구 노드
3. 호텔 예약 취소 및 변경을 위한 또 다른 도구 노드
   
이 그래프는 현재 상태에 기반하여 다음 단계를 결정하기 위해 조건부 엣지를 사용하여 동적이고 반응적인 워크플로우를 가능하게 합니다. 또한 상호작용 간에 상태를 유지하기 위한 메모리 관리도 설정할 것입니다.

In [ ]:
from langgraph.graph import END, StateGraph, MessagesState
from IPython.display import Image, display

# Create a new graph workflow
hotel_workflow = StateGraph(MessagesState)

hotel_workflow.add_node("hotel_agent", hotel_agent)
hotel_workflow.add_node("search_and_retrieve_node", search_and_retrieve_node)
hotel_workflow.add_node("change_and_cancel_node", change_and_cancel_node)

hotel_workflow.add_edge(START, "hotel_agent")

# We now add a conditional edge
hotel_workflow.add_conditional_edges(
    "hotel_agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    {
        # If agent decides to use `suggest_hotels` or  `retrieve_hotel_booking`
        "continue": "search_and_retrieve_node",
        # If agent decides to use `change_hotel_booking` or  `cancel_hotel_booking`
        "human_approval": "change_and_cancel_node",
        "end": END,
    },
)

hotel_workflow.add_edge("search_and_retrieve_node", "hotel_agent")
hotel_workflow.add_edge("change_and_cancel_node", "hotel_agent")

# Set up memory
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

hotel_graph_compiled = hotel_workflow.compile(
    checkpointer=memory, interrupt_before=["change_and_cancel_node"]
)

display(Image(hotel_graph_compiled.get_graph().draw_mermaid_png()))

In [ ]:
def _print_event(event: tuple, _printed: set, max_length=1500):
    event_dict = event[1] if isinstance(event, tuple) else event

    # Handle dialog state
    current_state = event_dict.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])

    message = event_dict.get("messages")
    if message:
        if not isinstance(message, list):
            message = [message]

        # Get the last message
        last_message = message[-1]

        message_id = getattr(last_message, "id", str(id(last_message)))

        if message_id not in _printed:
            # Handle pretty printing based on message type
            if hasattr(last_message, "pretty_repr"):
                msg_repr = last_message.pretty_repr(html=True)
            else:
                msg_repr = f"Content: {last_message.content}"
                if hasattr(last_message, "additional_kwargs"):
                    msg_repr += f"\nAdditional info: {last_message.additional_kwargs}"

            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"

            print(msg_repr)
            _printed.add(message_id)

In [ ]:
import uuid
from langchain_core.messages import ToolMessage

thread_id = str(uuid.uuid4())

_printed = set()
config = {"configurable": {"thread_id": thread_id}}

events = hotel_graph_compiled.stream(
    {"messages": ("user", "Get details of my booking id 203")},
    config,
    stream_mode="values",
)
for event in events:
    _print_event(event, _printed)

In [ ]:
thread_id = str(uuid.uuid4())

_printed = set()
config = {"configurable": {"thread_id": thread_id}}

events = hotel_graph_compiled.stream(
    {"messages": ("user", "cancel my hotel booking id 206")},
    config,
    stream_mode="values",
)
for event in events:
    _print_event(event, _printed)

In [ ]:
user_input = input(
    "Do you approve of the above actions? Type 'y' to continue;"
    " otherwise, explain your requested changed.\n\n"
)
if user_input.strip() == "y":
    # Just continue
    result = hotel_graph_compiled.invoke(
        None,
        config,
    )
    result["messages"][-1].pretty_print()
else:
    result = hotel_graph_compiled.invoke(
        {
            "messages": [
                ToolMessage(
                    tool_call_id=event["messages"][-1].tool_calls[0]["id"],
                    content=f"API call denied by user. Reasoning: '{user_input}'. Continue assisting, accounting for the user's input.",
                )
            ]
        },
        config,
    )
    result["messages"][-1].pretty_print()

## 서브그래프의 상태 업데이트

전체 그래프를 컴파일하면 호텔 에이전트가 서브그래프로 추가됩니다. 중단 후에 서브그래프의 상태를 업데이트해야 합니다.

이제 그래프를 테스트할 준비가 되었습니다. 메모리를 관리하기 위해 고유한 thread_id를 생성할 것입니다. 그래프를 테스트하기 위한 몇 가지 샘플 질문이 있습니다.

snapshot은 사용자 입력이 필요한 보류 중인 작업이나 결정이 있는지 확인하는 데 필요한 수퍼바이저 에이전트 그래프의 현재 상태를 검색합니다. while 루프 조건에서 snapshot.next 필드를 확인하여 사용자 승인이 필요한 보류 중인 작업이 있는지 확인합니다.

사용자 입력을 받은 후 상태를 업데이트합니다. 서브그래프의 상태를 업데이트할 때는 서브그래프의 config인 `state.tasks[0].state.config`를 전달해야 합니다.

사용자가 작업을 승인하면 상태를 업데이트하고 그래프를 호출합니다: `supervisor_agent_graph.invoke(None, config, subgraphs=True)`

사용자가 작업을 거부하면 도구 메시지로 상태를 업데이트한 다음 그래프를 계속 진행합니다.

In [ ]:
def extract_tool_id(pregel_task):
    # Navigate to the messages in the state
    messages = pregel_task.state.values.get("messages", [])

    # Find the last AIMessage
    for message in reversed(messages):
        if isinstance(message, AIMessage):
            # Check if the message has tool_calls
            tool_calls = getattr(message, "tool_calls", None)
            if tool_calls:
                # Return the id of the first tool call
                return tool_calls[0]["id"]

In [ ]:
thread_id = str(uuid.uuid4())
_printed = set()
config = {"configurable": {"thread_id": thread_id}}

questions = [
    "Get details of my flight booking id 200",
    "cancel my hotel booking id 136",
]
for question in questions:
    events = supervisor_agent_graph.stream(
        {"messages": ("user", question)}, config, stream_mode="values", subgraphs=True
    )
    for event in events:
        _print_event(event, _printed)
    snapshot = supervisor_agent_graph.get_state(config)
    while snapshot.next:
        try:
            user_input = input(
                "Do you approve of the above actions? Type 'y' to continue;"
                " otherwise, explain your requested changed.\n\n"
            )
        except:
            user_input = "y"
        if user_input.strip() == "y":
            # Just continue

            supervisor_agent_graph.update_state(
                state.tasks[0].state.config,
                {"messages": "Yes, cancel my booking"},
                as_node="change_and_cancel_node",
            )
            result = supervisor_agent_graph.invoke(None, config, subgraphs=True)
            result_dict = result[1]
            result_dict["messages"][-1].pretty_print()
        else:
            state = supervisor_agent_graph.get_state(config, subgraphs=True)
            tool_id = extract_tool_id(state.tasks[0])
            tool_message = [
                {
                    "tool_call_id": tool_id,
                    "type": "tool",
                    "content": f"API call denied by user. Reasoning: '{user_input}'. Continue assisting, accounting for the user's input.",
                }
            ]
            supervisor_agent_graph.update_state(
                state.tasks[0].state.config,
                {"messages": tool_message},
                as_node="change_and_cancel_node",
            )
            result = supervisor_agent_graph.invoke(None, config, subgraphs=True)
            _print_event(result, _printed)

        snapshot = supervisor_agent_graph.get_state(config)